#### Sistema de recomendación user-item:

- **def recomendacion_restaurante(user_id):**

> Ingresando el id de un usuario, deberíamos recibir una lista con 5 restaurantes recomendados para dicho usuario.

In [55]:
import pandas as pd

# Ruta del archivo
file_path = r'.\merged.csv'

# Leer el archivo en un DataFrame
reviews = pd.read_csv(file_path)


In [56]:
# Nos quedamos con las columnas relevantes
reviews = reviews[['name','category','rating','user_id']]

Primero vamos a filtrar solo las categorais de negocios relacionadas al rubro gastronomico

In [58]:
# Seleccionamos solo los valores únicos de categoría y los convertimos en cadenas de texto
categorias_unicas = reviews['category'].astype(str).unique()

# Convertimos el array de valores únicos en una lista de cadenas de texto
categorias_unicas = categorias_unicas.tolist()

import spacy

# Cargar el modelo en inglés de spaCy
nlp = spacy.load("en_core_web_md")

# Palabras clave a comparar
words = ['restaurant', 'food', 'cuisine']

# Inicializar una lista vacía para almacenar las categorías relacionadas
related_categories = []

# Iterar sobre cada categoría en categorias_unicas
for category in categorias_unicas:
    # Inicializar el puntaje de similaridad máximo
    max_similarity = 0.0
    # Iterar sobre cada palabra en words
    for word in words:
        # Calcular la similitud entre la categoría y la palabra
        similarity = nlp(category).similarity(nlp(word))
        # Actualizar el puntaje de similaridad máximo
        max_similarity = max(max_similarity, similarity)
    # Si el puntaje de similaridad máximo es mayor a un umbral, agregar la categoría a related_categories
    if max_similarity > 0.7:
        related_categories.append(category)

# Chequeamos resultado
related_categories


C:\Users\sebas\AppData\Local\Temp\ipykernel_39356\1132569886.py:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = nlp(category).similarity(nlp(word))


['Restaurant', 'Seafood restaurant', 'Pizza restaurant', 'Cuban restaurant', 'Health food store', 'Italian restaurant', 'Food and drink', 'Chinese restaurant', 'Taco restaurant', 'Wholesale food store', 'Fast food restaurant', 'Mexican restaurant', 'Mediterranean restaurant', 'Hamburger restaurant', 'Haitian restaurant', 'Sushi restaurant', 'Vegetarian restaurant', 'Jamaican restaurant', 'Vietnamese restaurant', 'Breakfast restaurant', 'Honduran restaurant', 'Health food restaurant', 'Cheesesteak restaurant', 'Barbecue restaurant', 'Sushi takeaway', 'Dominican restaurant', 'Colombian restaurant', 'Thai restaurant', 'Turkish restaurant', 'Caribbean restaurant', 'Soul food restaurant', 'Brazilian restaurant', 'Kosher restaurant', 'Japanese restaurant', 'Restaurant supply store', 'Authentic Japanese restaurant', 'Catering food and drink supplier', 'Buffet restaurant', 'Grill store', 'Hawaiian restaurant', 'Spanish restaurant', 'Organic restaurant', 'Chicken restaurant', 'Peruvian restaura

In [61]:
# Nos quedamos solo con las filas cuya categoría esté en related_categories
reviews = reviews[reviews['category'].isin(related_categories)]

In [63]:
reviews.shape

(88219, 4)

In [82]:
# Vamos a exportar e importar el dataset
reviews.to_csv('data_user_item.csv', index=False)

reviews = pd.read_csv('./data_user_item.csv')

In [64]:
# Eliminamos la columna category 
reviews.drop('category', axis=1, inplace=True)

# Renombramos name como business_id
reviews.rename(columns={'name': 'business_id'}, inplace=True)

C:\Users\sebas\AppData\Local\Temp\ipykernel_39356\2767750060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.drop('category', axis=1, inplace=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_39356\2767750060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.rename(columns={'name': 'business_id'}, inplace=True)


In [66]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV

# Convertir el tipo de datos de las columnas user_id y business_id a string
reviews['user_id'] = reviews['user_id'].astype(str)
reviews['business_id'] = reviews['business_id'].astype(str)

# Especificamos el valor mínimo y el valor máximo de los ratings y el dataset a utilizar
reader = Reader(rating_scale=(reviews["rating"].min(), reviews["rating"].max()))

# El objeto Dataset de Surprise nos permite leer datos 
dataset = Dataset.load_from_df(reviews[['user_id', 'business_id', 'rating']], reader)

# Empleamos GridSearchCV con el algoritmo SVD y seteamos el parámetro refit a True con measures = ["rmse","fcp"]
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

gs = GridSearchCV(SVD, param_grid, measures=['fcp', "rmse"], cv=3, refit=True)

# Entrenamos el modelo
gs.fit(dataset)


C:\Users\sebas\AppData\Local\Temp\ipykernel_39356\1290439482.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['user_id'] = reviews['user_id'].astype(str)
C:\Users\sebas\AppData\Local\Temp\ipykernel_39356\1290439482.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['business_id'] = reviews['business_id'].astype(str)


In [67]:
# Mejor combinacion de parametros
gs.best_params


{'fcp': {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4},
 'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

In [68]:
# Mejor Score
gs.best_score

{'fcp': 0.5073867032101574, 'rmse': 1.2477929435845014}

In [69]:
# Guardamos en una variable el modelo con mayor fcp
best_model = gs.best_estimator["fcp"]

In [71]:
reviews.head()

,business_id,rating,user_id
592,Bachata Rosa,5,1.1555531733618288e+20
593,Bachata Rosa,5,1.168683985741959e+20
594,Bachata Rosa,1,1.0898020753291964e+20
788,Zampini's Bottega,5,1.047022032410506e+20
789,Zampini's Bottega,5,1.0177046793955536e+20


In [73]:
# Prediccion para user_id : "1.0147185615514871e+20",  item_id : "0x8893863ea87bd5dd:0x9383ebf973e74abb

user_id = "1.1555531733618288e+20"
business_id = "Bachata Rosa"

pred = best_model.predict(user_id, business_id)

pred.est

4.168100271877366

In [74]:
from surprise.dump import dump

# Nombre del archivo para guardar el modelo
modelo_guardado = "modelo_user_item.pkl"

# Guardar el modelo entrenado
dump(modelo_guardado, algo=best_model)


In [77]:
def recomendacion_usuario(user_id, top_n=5):
    # Cargar el modelo entrenado desde el archivo guardado
    modelo_guardado = "modelo_user_item.pkl"
    loaded_model = load(modelo_guardado)[1]

    # Crear una lista vacía para almacenar las recomendaciones
    recomendaciones = []

    # Obtener todos los ítems que el usuario aún no ha calificado
    items_no_vistos = reviews[~reviews['business_id'].isin(reviews[reviews['user_id'] == user_id]['business_id'])]['business_id'].unique()

    # Realizar predicciones para cada ítem no visto
    for item_id in items_no_vistos:
        pred = loaded_model.predict(user_id, item_id)
        recomendaciones.append(item_id)

    # Ordenar las recomendaciones en orden descendente según la calificación estimada
    recomendaciones.sort(key=lambda x: loaded_model.predict(user_id, x).est, reverse=True)

    # Devolver los top N elementos recomendados
    return recomendaciones[:top_n]


In [80]:
recomendacion_usuario("1.047022032410506e+20")

['Dina Bella Cuban Cuisine',
 'Julianos BBQ',
 'StreetWise Urban Food',
 'Deccan Spice Pompano',
 'Girl And A Grill']